In [ ]:
import numpy as np

def load_matrix_txt(path: str):

    H = np.loadtxt(path, dtype=int)
    H = H % 2
    if H.ndim == 1:
        # If a single row is loaded, make it 2D
        H = H.reshape(1, -1)
    return H


def load_vector_txt(path: str):
 
    y = np.loadtxt(path, dtype=int)
    y = np.asarray(y).reshape(-1)  # flatten
    y = y % 2
    return y


def syndrome(H: np.ndarray, x: np.ndarray):
    
    return (H @ x) % 2



In [ ]:
# LDPC decoder (Loopy BP / Sum-Product) for BSC in LLR domain
def ldpc_decode_bsc(H_hat, y, p, max_iter):

    H = (H_hat.astype(int) % 2)
    y = (np.asarray(y).astype(int).reshape(-1) % 2)

    M, N = H.shape
    if y.shape[0] != N:
        raise ValueError(f"y has length {y.shape[0]} but H has {N} columns.")

    # Build adjacency lists
    check_neighbors = [np.where(H[m, :] == 1)[0] for m in range(M)]  # vars per check
    var_neighbors   = [np.where(H[:, i] == 1)[0] for i in range(N)]  # checks per var

    # For fast index mapping: pos_in_check[m][i] = position of variable i in check_neighbors[m]
    pos_in_check = [dict() for _ in range(M)]
    for m in range(M):
        for k, i in enumerate(check_neighbors[m]):
            pos_in_check[m][i] = k

    # Channel LLRs for BSC:
    # Lch_i = log( P(y_i|x_i=0) / P(y_i|x_i=1) )
    # If y_i=0: Lch = log((1-p)/p), if y_i=1: Lch = -log((1-p)/p)
    L0 = np.log((1.0 - p) / p)
    Lch = np.where(y == 0, L0, -L0).astype(float)

    # Messages stored per edge in "check-major" format:
    # Lq[m][k] = LLR message variable->check along edge (check m, its k-th neighbor variable)
    # Lr[m][k] = LLR message check->variable along same edge
    Lq = [Lch[check_neighbors[m]].copy() for m in range(M)]
    Lr = [np.zeros(len(check_neighbors[m]), dtype=float) for m in range(M)]

    # Iterations
    x_hat = np.zeros(N, dtype=int)

    # small helper for numeric safety
    def clamp_unit(v, eps=1e-12):
        return np.clip(v, -1.0 + eps, 1.0 - eps)

    for it in range(1, max_iter + 1):

        # (1) Check-to-variable update (Lr)
        # Lr = 2 * atanh( prod_{j≠i} tanh(Lq/2) )
        for m in range(M):
            deg = len(check_neighbors[m])
            if deg == 0:
                continue

            t = np.tanh(Lq[m] / 2.0)  # shape (deg,)
            total_prod = np.prod(t)

            for k in range(deg):
                # product excluding k
                if abs(t[k]) > 1e-15:
                    prod_excl = total_prod / t[k]
                else:
                    # fallback: compute explicitly excluding k to avoid divide by ~0
                    prod_excl = np.prod(np.delete(t, k))

                prod_excl = clamp_unit(prod_excl)
                Lr[m][k] = 2.0 * np.arctanh(prod_excl)

        # (2) Variable-to-check update (Lq)
        # Lq_{i->m} = Lch_i + sum_{m'∈N(i)\{m}} Lr_{m'->i}
        for i in range(N):
            checks = var_neighbors[i]
            if checks.size == 0:
                continue

            # sum of incoming Lr from all checks to variable i
            incoming_sum = 0.0
            for m in checks:
                k = pos_in_check[m][i]
                incoming_sum += Lr[m][k]

            # update each outgoing message to each check
            for m in checks:
                k = pos_in_check[m][i]
                Lq[m][k] = Lch[i] + (incoming_sum - Lr[m][k])

        # (3) Compute posterior LLRs and hard decision
        # Lpost_i = Lch_i + sum_{m∈N(i)} Lr_{m->i}
        # decide x_hat_i = 0 if Lpost>0 else 1
        for i in range(N):
            Lpost = Lch[i]
            for m in var_neighbors[i]:
                k = pos_in_check[m][i]
                Lpost += Lr[m][k]
            x_hat[i] = 1 if (Lpost < 0) else 0

        # (4) Stop if syndrome is zero
        if np.all(syndrome(H, x_hat) == 0):
            return x_hat, 0, it

    return x_hat, -1, max_iter


# Run the required experiment
def run_assignment3(H_path="H1.txt", y_path="y1.txt", p=0.1, max_iter=20):
    H1 = load_matrix_txt(H_path)
    y1 = load_vector_txt(y_path)

    x_hat, code, iters = ldpc_decode_bsc(H1, y1, p, max_iter=max_iter)

    print("Decoded vector x_hat:")
    print(x_hat.tolist())
    print("Return code:", code, "(0=success, -1=not converged)")
    print("Iterations used:", iters)
    return x_hat, code, iters


if __name__ == "__main__":
    run_assignment3("H1.txt", "y1.txt", p=0.1, max_iter=20)


Decoded vector x_hat:
[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,

In [ ]:
x_hat, code, iters = run_assignment3("H1.txt", "y1.txt", p=0.1, max_iter=20)


def bits_to_ascii(bits, msb_first=True):
    bits = list(bits)
    out = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if not msb_first:
            byte = byte[::-1]
        val = int("".join(map(str, byte)), 2)
        out.append(chr(val))
    return "".join(out)

bits_248 = x_hat[:248]

msg = bits_to_ascii(bits_248, msb_first=True)
print("Decoded message (MSB-first):", repr(msg))

msg2 = bits_to_ascii(bits_248, msb_first=False)
print("Decoded message (LSB-first):", repr(msg2))


Decoded vector x_hat:
[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,